In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for extract_filter_atl08.py
using the rebinned 30m h5 granules

In [2]:
from os import path
import os, glob
import datetime
import pandas as pd
import geopandas as gpd

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
curr_dir = wk_dir = os.path.dirname(os.path.realpath('__file__'))
print(curr_dir)

/projects/icesat2_boreal/dps/alg_2-3


## Test extract_filter_atl08.py code on Test Data
We are running extract_filter_atl08.py (but *FILTERING* is turned off):

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "https://maap-ops-workspace.s3.amazonaws.com.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python lib/extract_filter_atl08.py -i "path/to/h5file" -o "path/of/out/dir" --no-filter-qual --do_30m

### Build the input list of granules - this way is veeeeery slow
##### replace the local path with the https url

In [30]:
%%time
if False:
    INPUT_ATL08_GRANULE_LIST = glob.glob("/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/03/**/*.h5", recursive=True)
    INPUT_ATL08_GRANULE_LIST = [i.replace("/projects/my-private-bucket", "https://maap-ops-workspace.s3.amazonaws.com/lduncanson") for i in INPUT_ATL08_GRANULE_LIST]
    len(INPUT_ATL08_GRANULE_LIST)
    print(f"# rebinned (30m) ATL08 granules: {len(INPUT_ATL08_GRANULE_LIST)}")

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 8.11 µs


### Build the input list of granules - s3fs implementation

In [39]:
%%time
import s3fs
s3 = s3fs.S3FileSystem()
bucket = "s3://maap-ops-workspace"
searchkey_list = [f'lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/{DAY}/**/*.h5' for DAY in ['03','04','05']]
# Concat list of lists to data frame
atl0830m_path_df = pd.concat([pd.DataFrame(s3.glob(os.path.join(bucket, searchkey)), columns=['maap_path']) for searchkey in searchkey_list])

#Convert data frame to list
INPUT_ATL08_GRANULE_LIST = [i.replace("maap-ops-workspace", "https://maap-ops-workspace.s3.amazonaws.com") for i in atl0830m_path_df.maap_path.to_list()]

AttributeError: 'str' object has no attribute 'shape'

In [40]:
len(INPUT_ATL08_GRANULE_LIST)

47310

In [41]:
TEST_DPS  = False

if TEST_DPS:
    print('Running DPS on a SUBSET list of input')
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST[-10:]
else:
    print('Running DPS on the FULL list of input')
    DPS_INPUT_ATL08_GRANULE_LIST = INPUT_ATL08_GRANULE_LIST
    
DPS_INPUT_ATL08_GRANULE_LIST[-1]

Running DPS on the FULL list of input


'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/05/05/29/30/042845/ATL08_30m_20190504205925_05590305_005_01.h5'

## Run a DPS job across the list

In [42]:
job_results_list = []

for i, INPUT_ATL08_GRANULE in enumerate(DPS_INPUT_ATL08_GRANULE_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_extract_filter_atl08'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_param_dict = {
                        'input_file': INPUT_ATL08_GRANULE
                    }

    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_ATL08_GRANULE
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE
    job_results_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 100, 500, 1000, 3000, len(INPUT_ATL08_GRANULE_LIST)]:
        #print(f"DPS run #: {DPS_num} | granule name: {os.path.basename(INPUT_ATL08_GRANULE)} | job info: {submit_result}") 
        print(submit_result)
        
job_results_df = pd.concat(job_results_list)
job_results_df

{'status': 'success', 'http_status_code': 200, 'job_id': '28522faa-f1b2-489f-b578-36c445a9a159', 'dps_num': 1, 'tile_num': 'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/03/01/08/50/075739/ATL08_30m_20181116094717_07450103_005_01.h5', 'submit_time': datetime.datetime(2022, 3, 7, 20, 53, 53, 149896), 'dbs_job_hour': 20, 'algo_id': 'run_extract_filter_atl08_ubuntu', 'user': 'lduncanson', 'worker_type': 'maap-dps-worker-8gb'}
{'status': 'success', 'http_status_code': 200, 'job_id': 'c43d780f-2be8-441d-934b-67207ef8f354', 'dps_num': 100, 'tile_num': 'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/04/03/26/58/394017/ATL08_30m_20181020134416_03350106_005_01.h5', 'submit_time': datetime.datetime(2022, 3, 7, 20, 54, 7, 414435), 'dbs_job_hour': 20, 'algo_id': 'run_extract_filter_atl08_ubuntu', 'user': 'lduncanson', 'worker_type': 'maap-dps-worker-8gb'}
{'status': 'success', 'http_statu

,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,28522faa-f1b2-489f-b578-36c445a9a159,1,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.149896,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,ad3d5b38-61a8-4283-a2ae-63b827c309e1,2,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.243837,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,1ab341b7-c4c4-49e9-b6d3-ec5f9a8beb83,3,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.320869,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,4309a4e8-02a0-4ae8-9928-32ebfb883c4c,4,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.414337,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,dba94bc2-7ba4-4a1a-8164-7c8a653cb2b7,5,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.630910,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
...,...,...,...,...,...,...,...,...,...,...
0,success,200,37c79da4-7972-4ca0-a5c8-524f75509b3c,47306,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 00:05:29.844667,0,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,739b0400-f4e4-4fe7-85f8-3484c1f00783,47307,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 00:05:29.964277,0,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,acfb7b5c-26c3-4ceb-a60e-7dee0bf02e84,47308,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 00:05:30.171586,0,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,d4fbce73-51d5-4dcb-b78c-ef612483e031,47309,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 00:05:30.385272,0,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb


In [17]:
!pip install xmltodict

  Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)


In [39]:
pd.__version__

'1.2.2'

After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
### Build a table of job status based on job id - how many jobs failed?

In [146]:
%%time
def BUILD_TABLE_JOBSTATUS(job_results_df):
    import xmltodict
    #tmp_job_results_df = job_results_df#.iloc[47300:47310]
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in job_results_df.job_id.to_list()])
    return df

df = BUILD_TABLE_JOBSTATUS(job_results_df)
df

KeyboardInterrupt: 

#### Join job_results_df with job status info - which granules failed?

In [133]:
num_jobs_round1 = job_results_df.shape[0]
z = job_results_df.merge(df, how='left', left_on='job_id', right_on='wps:JobID')
fails_df = z[z['wps:Status'] =='Failed']
num_fails_round1 = fails_df.shape[0]
print(f'Count Round 1 total jobs:\t{num_jobs_round1}')
print(f'Count Round 1 failed jobs:\t{num_fails_round1}')
print(f'Count Round 1 jobs for Round 2:\t{num_jobs_round1-num_fails_round1}')

Count Round 1 total jobs:	47310
Count Round 1 failed jobs:	34767
Count Round 1 jobs for Round 2:	12543


##### Pick a job and manually test - does it produce output as expected?
if so, then AWS had some issue during the DPS: contact MAAP dev about this

In [112]:
f = fails_df.iloc[1].tile_num.replace('https://maap-ops-workspace.s3.amazonaws.com/lduncanson','/projects/my-private-bucket')
!python /projects/icesat2_boreal/dps/alg_2-3/extract_filter_atl08.py --i $f --no-filter-qual --do_30m -o /projects/my-public-bucket/test_output


Written by:
	Nathan Thomas	| @Nmt28
	Paul Montesano	| paul.m.montesano@nasa.gov

Min lat: 45.0
Max lat: 75.0
Min lon: -180.0
Max lon: 180.0
Month range: 6-9

ATL08 granule name: 	ATL08_30m_20181116003732_07390106_005_01
Input dir: 		/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/03/03/01/11/48/273239

Segment length: 30m
Find src nodata value using max of h_can: 	nan

Building pandas dataframe...
Setting pandas df nodata values to np.nan for some basic eval.
# of ATL08 obs: 		104613
# of ATL08 obs (can pho.>=0): 	24727
# of ATL08 obs (toc pho.>=0): 	24727
# of ATL08 obs (h_can>=0): 	24677
# of ATL08 obs (h_can<0): 	50
Setting out pandas df nodata values: 	3.4028234663852886e+38
Quality Filtering: 	[OFF] (do downstream)
Geographic Filtering: 	[ON] xmin = -180.0, xmax = 180.0, ymin = 45.0, ymax = 75.0
Creating CSV: 		/projects/my-public-bucket/test_output/ATL08_30m_20181116003732_07390106_005_01_30m.csv


### Compile granules of the fails for another DPS batch (Round2)

In [117]:
DPS_INPUT_ATL08_GRANULE_LIST_ROUND2 = fails_df.tile_num.to_list()
DPS_INPUT_ATL08_GRANULE_LIST_ROUND2[0]

'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/03/01/08/50/075739/ATL08_30m_20181116094717_07450103_005_01.h5'

In [118]:
#job_results_list_round2 = []

for i, INPUT_ATL08_GRANULE in enumerate(DPS_INPUT_ATL08_GRANULE_LIST_ROUND2):

    DPS_num = i+1
    IDENTIFIER = 'run_extract_filter_atl08'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_param_dict = {
                        'input_file': INPUT_ATL08_GRANULE
                    }

    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_ATL08_GRANULE
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE
    job_results_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 100, 500, 1000, 3000, len(INPUT_ATL08_GRANULE_LIST)]:
        #print(f"DPS run #: {DPS_num} | granule name: {os.path.basename(INPUT_ATL08_GRANULE)} | job info: {submit_result}") 
        print(submit_result)
        
job_results_df_round2 = pd.concat(job_results_list)
job_results_df_round2

{'status': 'success', 'http_status_code': 200, 'job_id': 'a0303acd-fe1d-4ade-affa-2326a6aca7f1', 'dps_num': 1, 'tile_num': 'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/03/01/08/50/075739/ATL08_30m_20181116094717_07450103_005_01.h5', 'submit_time': datetime.datetime(2022, 3, 8, 3, 5, 7, 982711), 'dbs_job_hour': 3, 'algo_id': 'run_extract_filter_atl08_ubuntu', 'user': 'lduncanson', 'worker_type': 'maap-dps-worker-8gb'}
{'status': 'success', 'http_status_code': 200, 'job_id': 'e5572773-2e49-42ef-8e00-d432f270c751', 'dps_num': 100, 'tile_num': 'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/04/03/27/08/099426/ATL08_30m_20181014100103_02410106_005_01.h5', 'submit_time': datetime.datetime(2022, 3, 8, 3, 5, 24, 329220), 'dbs_job_hour': 3, 'algo_id': 'run_extract_filter_atl08_ubuntu', 'user': 'lduncanson', 'worker_type': 'maap-dps-worker-8gb'}
{'status': 'success', 'http_status_code

NameError: name 'job_results_list_rounds2' is not defined

##### The round2 job results list was appended to the round1 list: build a table with all

In [136]:
job_results_df_all = pd.concat(job_results_list)
job_results_df_all

,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,28522faa-f1b2-489f-b578-36c445a9a159,1,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.149896,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,ad3d5b38-61a8-4283-a2ae-63b827c309e1,2,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.243837,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,1ab341b7-c4c4-49e9-b6d3-ec5f9a8beb83,3,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.320869,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,4309a4e8-02a0-4ae8-9928-32ebfb883c4c,4,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.414337,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,dba94bc2-7ba4-4a1a-8164-7c8a653cb2b7,5,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-07 20:53:53.630910,20,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
...,...,...,...,...,...,...,...,...,...,...
0,success,200,9d4a0558-7cef-4d4d-a631-f79f04cfed92,34763,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 05:49:44.692949,5,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,664bf7a9-6700-483c-8b2d-ecd9633bcb51,34764,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 05:49:44.886458,5,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,d86e313e-a298-450e-b127-94c3481258b6,34765,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 05:49:45.101102,5,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,ee687259-025c-4699-b967-7a13a5841f4b,34766,https://maap-ops-workspace.s3.amazonaws.com/ld...,2022-03-08 05:49:45.391512,5,run_extract_filter_atl08_ubuntu,lduncanson,maap-dps-worker-8gb


#### Update the job status table with job status results from Round 2

In [147]:
%%time
# The index of the last job from round 1: num_jobs_round1-1
# Confirm by checking the submit_time difference between the last round1 and the first round2 job
print(
    job_results_df_all.iloc[num_jobs_round1-1] ,
    job_results_df_all.iloc[num_jobs_round1]
)

job_status_df_all = BUILD_TABLE_JOBSTATUS(job_results_df_all)
job_status_df_all

status                                                        success
http_status_code                                                  200
job_id                           df9753b8-dd90-486b-8bbf-90eb1ad1a41c
dps_num                                                         47310
tile_num            https://maap-ops-workspace.s3.amazonaws.com/ld...
submit_time                                2022-03-08 00:05:30.819628
dbs_job_hour                                                        0
algo_id                               run_extract_filter_atl08_ubuntu
user                                                       lduncanson
worker_type                                       maap-dps-worker-8gb
Name: 0, dtype: object status                                                        success
http_status_code                                                  200
job_id                           a0303acd-fe1d-4ade-affa-2326a6aca7f1
dps_num                                                            

,@xmlns:ows,@xmlns:schemaLocation,@xmlns:wps,@xmlns:xsi,wps:JobID,wps:Status
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,28522faa-f1b2-489f-b578-36c445a9a159,Failed
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,ad3d5b38-61a8-4283-a2ae-63b827c309e1,Failed
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,1ab341b7-c4c4-49e9-b6d3-ec5f9a8beb83,Failed
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,4309a4e8-02a0-4ae8-9928-32ebfb883c4c,Failed
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,dba94bc2-7ba4-4a1a-8164-7c8a653cb2b7,Failed
...,...,...,...,...,...,...
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,9d4a0558-7cef-4d4d-a631-f79f04cfed92,Succeeded
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,664bf7a9-6700-483c-8b2d-ecd9633bcb51,Failed
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,d86e313e-a298-450e-b127-94c3481258b6,Succeeded
wps:StatusInfo,http://www.opengis.net/ows/2.0,http://schemas.opengis.net/wps/2.0/wps.xsd,http://www.opengis.net/wps/2.0,http://www.w3.org/2001/XMLSchema-instance,ee687259-025c-4699-b967-7a13a5841f4b,Succeeded


In [151]:
job_results_df_all.iloc[num_jobs_round1:].shape

(34767, 10)

##### Isolate the granules associated with the Round 2 fails from the updated 'all' job status table

In [156]:
j_results_status = job_results_df_all.iloc[num_jobs_round1:].merge(job_status_df_all, how='left', left_on='job_id', right_on='wps:JobID')
fails_df_round2 = j_results_status[j_results_status['wps:Status'] =='Failed']
num_fails_round2 = fails_df_round2.shape[0]
print(f'Count total jobs:\t{num_jobs_round1}')
print(f'Count Round 2 failed jobs:\t{num_fails_round2}')

Count total jobs:	47310
Count Round 2 failed jobs:	24524


### Compile granules from Round 2 fails for a new DPS batch

In [158]:
DPS_INPUT_ATL08_GRANULE_LIST_ROUND3 = fails_df_round2.tile_num.to_list()
print(len(DPS_INPUT_ATL08_GRANULE_LIST_ROUND3))
DPS_INPUT_ATL08_GRANULE_LIST_ROUND3[0]


24524


'https://maap-ops-workspace.s3.amazonaws.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/03/03/01/08/50/075739/ATL08_30m_20181116094717_07450103_005_01.h5'

### Examine output of DPS - but replace glob.glob with s3.glob

In [19]:
#print(f"The data frame show you submitted {len(job_results_df)} jobs. Check the returned results to see if the total returned = total submitted...")
for JOB_HOUR in range(20,21):
    returned_results_list = glob.glob(f"/projects/my-private-bucket/dps_output/run_extract_filter_atl08_ubuntu/master/2022/03/04/{JOB_HOUR}/**/*.csv", recursive=True)
    print(f"For DPS job that returned results in hour {JOB_HOUR}, # granules that ran: {len(returned_results_list)}")

For DPS job that returned results in hour 20, # granules that ran: 10


In [32]:
# Merge all files in the list
print("Creating pandas data frame...")
atl08_gdf = pd.concat((pd.read_csv(f) for f in returned_results_list ), sort=False, ignore_index=True) # <--generator is (), list is []
atl08_gdf = gpd.GeoDataFrame(atl08_gdf, geometry=gpd.points_from_xy(atl08_gdf.lon, atl08_gdf.lat), crs='epsg:4326')

Creating pandas data frame...


In [34]:
atl08_gdf.head()

,fid,lon,lat,dt,orb_orient,orb_num,rgt,gt,segid_beg,segid_end,...,asr,h_dif_ref,ter_flg,ph_rem_flg,dem_rem_flg,seg_wmask,lyr_flg,seg_cover,granule_name,geometry
0,14887,75.620664,45.000490,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249813.842915,249817.842915,...,0.158199,-1.342682,1,0,0,0,0,48,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62066 45.00049)
1,14888,75.620628,45.000758,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249815.336919,249819.336919,...,0.158199,-1.342682,1,0,0,0,0,48,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62063 45.00076)
2,14889,75.620401,45.002371,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249824.331298,249828.331298,...,0.158199,-1.073212,1,0,0,0,0,42,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62040 45.00237)
3,14890,75.620364,45.002640,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249825.830880,249829.830880,...,0.158199,-1.073212,1,0,0,0,0,42,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62036 45.00264)
4,14891,75.620291,45.003178,b'2019-04-08T02:04:27.000000Z',0,3125,150,b'gt1r',249828.830880,249832.830880,...,0.129356,-1.743439,1,0,0,0,0,24,ATL08_30m_20190408015557_01500302_005_01.h5,POINT (75.62029 45.00318)


In [27]:
os.system( "python /projects/icesat2_boreal/lib/build_tindex_master.py -t ATL08 -dps_year 2022 -m 3 --start_day '04' --outdir /projects/my-public-bucket/test_output")

512